# faceRec

This IPython notebook will recognize a face in a video


```

| Yale         | JAFFE        | FERG         | CK+          |
| ------------ | ------------ | ------------ | ------------ |
| 15 subjects  | 60 subjects  | 6  subjects  | 123 subjects |
| 1 female     | 60 female    | 3  female    | 85 female    |
| 1 image each | variable #   | variable #   | variable #   | 
| ------------ | ------------ | ------------ | ------------ |
| happy        | happy        | happy        | happy        |
| sad          | sad          | sad          | sad          |
| surprise     | surprise     | surprise     | surprise     |
| neutral      |              | neutral      | neutral      |
|              | anger        | anger        | anger        |
|              | disgust      | disgust      | disgust      |
|              | fear         | fear         | fear         |


CK+ Notes:

4) The Emotion coded files (Emotion_labels.zip) - ONLY 327 of the 593 sequences have emotion sequences. This is because these are the only ones the fit the prototypic definition. Like the FACS files, there is only 1 Emotion file for each sequence which is the last frame (the peak frame). There should be only one entry and the number will range from 0-7 (i.e. 0=neutral, 1=anger, 2=contempt, 3=disgust, 4=fear, 5=happy, 6=sadness, 7=surprise). N.B there is only 327 files- IF THERE IS NO FILE IT MEANS THAT THERE IS NO EMOTION LABEL (sorry to be explicit but this will avoid confusion).


JAFFE Notes:

-------------- Semantic Ratings Data (Fear Excluded) --------------

This rating experiment excluded fear images and and the fear adjective
from the ratings. We did this because we thought that the expressors
were not good at posing fear. There is some evidence in the scientific
literature that fear may be processed differently from the other basic
facial expressions.

Please note that there are less subjects than the other experiment.

```

# CK+ images

In [121]:
import cv2
import glob
import os

# OpenCV face classifier
cascPath = '/usr/local/Cellar/opencv/2.4.13.2/share/OpenCV/haarcascades/haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(cascPath)

# Set paths, variable values
dbDir = '/Users/abock/Desktop/InsightProject/faceDatabases/CK+/'
emoDir = '/Users/abock/Desktop/InsightProject/faceDatabases/CK+/Emotion/'
emotions = ['anger','disgust','fear','joy','neutral','sadness','surprise']
emoNums = [1,3,4,5,0,6,7]


subjectNames = []
fileNames = []
subjectEmotions = []

# Find the images and emotion labels
for subDir in glob.glob(emoDir + 'S*/'):
    subject = os.path.basename(os.path.dirname(subDir))
    subjectNames.append(subject)
    for faceDir in glob.glob(subDir + '*/'):
        dirNum = os.path.basename(os.path.dirname(faceDir))
        for thisFile in glob.glob(faceDir + '*.txt'):
            imDir = dbDir + subject + dirNum
            imFile = os.path.basename(thisFile[:-12])
            fileNames.append(imFile + '.png')
            with open(thisFile) as f:
                for line in f:
                    line = map(float, line.split())
                    subjectEmotions.append(line)
                    
# Loop through images
for inFace in glob.glob(inDir + '*.png'):
    # Load the image
    rawImg = cv2.imread(inFace)

    # Detect faces in the image
    faces = faceCascade.detectMultiScale(
    rawImg,
    scaleFactor=scaleSize,
    minNeighbors=numNeighbors,
    minSize=(faceSize, faceSize),
    flags = cv2.CASCADE_SCALE_IMAGE
    )

    # If a face is detected
    if len(faces):
        # Convert to gray
        gImg = cv2.cvtColor(rawImg, cv2.COLOR_BGR2GRAY)

        # Resize just the face
        x,y,w,h = faces[0,:]
        faceImg = gImg[y:y+h,x:x+w]
        img = cv2.resize(faceImg,(outImgSize,outImgSize),interpolation = cv2.INTER_CUBIC)
        # Flatten
        l = img.shape[0] * img.shape[1]
        img = img.reshape(1, l)

        # Append to output matrices
        trainImages.append(img[0])
        trainSubs.append(subName)
        trainEmotions.append(emoName)
            
trainImages = np.asarray(trainImages)
trainSubs = np.asarray(trainSubs)
trainEmotions = np.asarray(trainEmotions)

cv2.destroyAllWindows()
print("done in %0.3fs" % (time() - t0))
    

327

In [90]:
import numpy as np
with open(thisFile) as f:
    for line in f:
        line = map(float, line.split())

array([ 4.])

In [35]:
import os
foo1 = os.path.dirname(faceDir)
foo2 = os.path.basename(foo1)
print(foo1)
print(foo2)

/Users/abock/Desktop/InsightProject/faceDatabases/CK+/Emotion/S999/003
003


# Import the required modules

In [ ]:
# import the required modules
import cv2
import numpy
import math
import os

In [ ]:
# Get user supplied values
imagePath = '/Users/abock/Downloads/Barack_Obama_family_portrait_2011.jpg'
cascPath = '/usr/local/Cellar/opencv/2.4.13.2/share/OpenCV/haarcascades/haarcascade_frontalface_default.xml'

In [ ]:
# Create the haar cascade
faceCascade = cv2.CascadeClassifier(cascPath)

### Load one image

In [ ]:
import cv2
import numpy
import time
inDir = '/Users/abock/Desktop/InsightProject/'
cascPath = '/usr/local/Cellar/opencv/2.4.13.2/share/OpenCV/haarcascades/haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(cascPath)

rawImg = cv2.imread(inDir + 'sad_keanu_Big.png')
h,w = rawImg.shape[:2]
#imS = cv2.resize(rawImg, (w*2,h*2),interpolation = cv2.INTER_CUBIC)
imS = rawImg
gray = cv2.cvtColor(imS, cv2.COLOR_BGR2GRAY)

# Detect faces in the image
faces = faceCascade.detectMultiScale(
    imS,
    scaleFactor=1.1,
    minNeighbors=5,
    minSize=(250, 250),
    flags = cv2.CASCADE_SCALE_IMAGE
    )
for (x, y, w, h) in faces:
    cv2.rectangle(imS, (x, y), (x+w, y+h), (0, 255, 0), 4)
    cv2.imshow('Face',imS)
    cv2.waitKey(30)

time.sleep(3)
cv2.destroyAllWindows()

### Load all faces

In [340]:
import cv2
import numpy as np
import os
import glob
from sklearn.neighbors import NearestNeighbors
import pylab
from time import time

cv2.destroyAllWindows()

# Face defaults
cascPath = '/usr/local/Cellar/opencv/2.4.13.2/share/OpenCV/haarcascades/haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(cascPath)

# Set paths, variable values
faceDir = '/Users/abock/Desktop/InsightProject/FERG_DB_256/'
trainNames = ['aia','bonnie','jules','malcolm','mery']
# emotions = ['anger','disgust','fear','joy','neutral','sadness','surprise']
emotions = ['anger','disgust','fear']

# Face processing defaults
outImgSize = 100
scaleSize = 1.05
numNeighbors = 3
faceSize = 150

print("Loading input images...")
t0 = time()

# Loop through all images in the input directory
trainImages = []
trainSubs = []
trainEmotions = []
for subName in trainNames:
    # Subject name
    print('Subject = ' + subName)
    
    for emoName in emotions:
        # Emotion name
        print(emoName)
        
        # Input directory
        inDir = ''.join((faceDir,subName,'/',subName,'_',emoName,'/'))
        
        for inFace in glob.glob(inDir + '*.png'):
            # Load the image
            rawImg = cv2.imread(inFace)
            
            # Detect faces in the image
            faces = faceCascade.detectMultiScale(
            rawImg,
            scaleFactor=scaleSize,
            minNeighbors=numNeighbors,
            minSize=(faceSize, faceSize),
            flags = cv2.CASCADE_SCALE_IMAGE
            )
            
            # If a face is detected
            if len(faces):
                # Convert to gray
                gImg = cv2.cvtColor(rawImg, cv2.COLOR_BGR2GRAY)
            
                # Resize just the face
                x,y,w,h = faces[0,:]
                faceImg = gImg[y:y+h,x:x+w]
                img = cv2.resize(faceImg,(outImgSize,outImgSize),interpolation = cv2.INTER_CUBIC)
                # Flatten
                l = img.shape[0] * img.shape[1]
                img = img.reshape(1, l)
            
                # Append to output matrices
                trainImages.append(img[0])
                trainSubs.append(subName)
                trainEmotions.append(emoName)
            
trainImages = np.asarray(trainImages)
trainSubs = np.asarray(trainSubs)
trainEmotions = np.asarray(trainEmotions)

cv2.destroyAllWindows()
print("done in %0.3fs" % (time() - t0))

Loading input images...
Subject = aia
anger
disgust
fear
Subject = bonnie
anger
disgust
fear
Subject = jules
anger
disgust
fear
Subject = malcolm
anger
disgust
fear
Subject = mery
anger
disgust
fear
done in 460.748s


In [316]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(
#     trainImages, trainEmotions, test_size=0.25, random_state=42)

In [374]:
from sklearn.decomposition import PCA
from time import time 

X_train = trainImages
Y_train = trainEmotions

# n_components = len(emotions)
n_components = 6

print("Extracting the top %d eigenfaces from %d faces"
      % (n_components, X_train.shape[0]))
t0 = time()
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(X_train)
print("done in %0.3fs" % (time() - t0))

Extracting the top 6 eigenfaces from 19614 faces
done in 11.693s


In [375]:
print("Projecting the input data on the eigenfaces orthonormal basis")
t0 = time()
X_train_pca = pca.transform(X_train)
print("done in %0.3fs" % (time() - t0))

Projecting the input data on the eigenfaces orthonormal basis
done in 1.640s


In [378]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import neighbors

print("Fitting the classifier to the training set")
t0 = time()
# param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
#               'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
#clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)

metrics = ['minkowski','euclidean','manhattan'] 
weights = ['uniform','distance'] #10.0**np.arange(-5,4)
numNeighbors = np.arange(1,30)
algorithms = ['ball_tree', 'kd_tree', 'brute']
param_grid = dict(metric=metrics,weights=weights,n_neighbors=numNeighbors,algorithm=algorithms)
clf = GridSearchCV(neighbors.KNeighborsClassifier(), param_grid)

clf = clf.fit(X_train_pca, Y_train)
print("done in %0.3fs" % (time() - t0))
print("Best estimator found by grid search:")
print(clf.best_estimator_)

Fitting the classifier to the training set


KeyboardInterrupt: 

In [377]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

X_test = testImages
Y_test = testEmotions

X_test_pca = pca.transform(X_test)
Y_pred = clf.predict(X_test_pca)
print(classification_report(Y_test, Y_pred, target_names=emotions))

             precision    recall  f1-score   support

      anger       0.48      1.00      0.65      1459
    disgust       0.17      0.00      0.01      1465
       fear       1.00      0.87      0.93      1023

avg / total       0.50      0.60      0.48      3947



In [355]:
Y_test.shape

AttributeError: 'list' object has no attribute 'shape'

In [325]:
y_test[100]

'sadness'

In [353]:
y_pred[100]

'fear'

In [352]:
cv2.destroyAllWindows()

# Face defaults
cascPath = '/usr/local/Cellar/opencv/2.4.13.2/share/OpenCV/haarcascades/haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(cascPath)

# Set paths, variable values
faceDir = '/Users/abock/Desktop/InsightProject/FERG_DB_256/'
# trainNames = ['aia','bonnie','jules','malcolm','mery']
trainNames = ['aia','malcolm']
testNames = ['ray']
#emotions = ['anger','disgust','fear','joy','neutral','sadness','surprise']
emotions = ['anger','disgust','fear']

# Face processing defaults
outImgSize = 100
scaleSize = 1.05
numNeighbors = 3
faceSize = 150

print("Loading input images...")
t0 = time()

# Loop through all images in the input directory
testImages = []
testSubs = []
testEmotions = []
for subName in testNames:
    # Subject name
    print('Subject = ' + subName)
    
    for emoName in emotions:
        # Emotion name
        print(emoName)
        
        # Input directory
        inDir = ''.join((faceDir,subName,'/',subName,'_',emoName,'/'))
        
        for inFace in glob.glob(inDir + '*.png'):
            # Load the image
            rawImg = cv2.imread(inFace)
            
            # Detect faces in the image
            faces = faceCascade.detectMultiScale(
            rawImg,
            scaleFactor=scaleSize,
            minNeighbors=numNeighbors,
            minSize=(faceSize, faceSize),
            flags = cv2.CASCADE_SCALE_IMAGE
            )
            
            # If a face is detected
            if len(faces):
                # Convert to gray
                gImg = cv2.cvtColor(rawImg, cv2.COLOR_BGR2GRAY)
            
                # Resize just the face
                x,y,w,h = faces[0,:]
                faceImg = gImg[y:y+h,x:x+w]
                img = cv2.resize(faceImg,(outImgSize,outImgSize),interpolation = cv2.INTER_CUBIC)
                # Flatten
                l = img.shape[0] * img.shape[1]
                img = img.reshape(1, l)
            
                # Append to output matrices
                testImages.append(img[0])
                testSubs.append(subName)
                testEmotions.append(emoName)
            
testImages = np.asarray(testImages)
testSubs = np.asarray(testSubs)
testEmotions = np.asarray(testEmotions)

cv2.destroyAllWindows()
print("done in %0.3fs" % (time() - t0))

Loading input images...
Subject = ray
anger
disgust
fear
done in 134.411s


In [337]:
print(testEmotions)
print(y_pred)

['anger' 'anger' 'anger' ..., 'surprise' 'surprise' 'surprise']
['surprise' 'surprise' 'surprise' ..., 'surprise' 'surprise' 'surprise']


In [339]:
X_test_pca = pca.transform(testImages)
y_pred = clf.predict(X_test_pca)
print(classification_report(testEmotions, y_pred, target_names=emotions))

             precision    recall  f1-score   support

      anger       1.00      0.24      0.39      2042
    disgust       0.00      0.00      0.00      2088
       fear       0.63      0.16      0.25      1255
        joy       0.00      0.00      0.00      1438
    neutral       0.00      0.00      0.00      1243
    sadness       0.15      1.00      0.26      1506
   surprise       0.00      0.00      0.00      1681

avg / total       0.27      0.20      0.13     11253



### Show cropped face

In [ ]:
import cv2
import numpy
import time
inDir = '/Users/abock/Desktop/InsightProject/'
cascPath = '/usr/local/Cellar/opencv/2.4.13.2/share/OpenCV/haarcascades/haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(cascPath)

rawImg = cv2.imread(inDir + 'happy-person.png')
h,w = rawImg.shape[:2]
#imS = cv2.resize(rawImg, (w*2,h*2),interpolation = cv2.INTER_CUBIC)
imS = rawImg
gray = cv2.cvtColor(imS, cv2.COLOR_BGR2GRAY)

# Detect faces in the image
faces = faceCascade.detectMultiScale(
    imS,
    scaleFactor=1.1,
    minNeighbors=1,
    minSize=(200, 200),
    flags = cv2.CASCADE_SCALE_IMAGE
    )
for (x, y, w, h) in faces:
    cv2.rectangle(imS, (x, y), (x+w, y+h), (0, 255, 0), 4)
    face = imS[y:y+h,x:x+w]
    cv2.imshow('Face',face)
    cv2.waitKey(30)

time.sleep(3)
cv2.destroyAllWindows()

In [ ]:
gray

# Demo movie

In [ ]:
import cv2
import numpy
import glob

inDir = '/Users/abock/Desktop/InsightProject/FERG_DB_256/aia/aia_joy/'
cascPath = '/usr/local/Cellar/opencv/2.4.13.2/share/OpenCV/haarcascades/haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(cascPath)

allFiles = glob.glob(inDir + '*png')

# Define the codec and create VideoWriter object
outFile = '/Users/abock/Desktop/aia_joy.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')

vid = None
is_color = True
ct = 0
for inFace in allFiles:
    ct = ct + 1
    rawImg = cv2.imread(inFace)
    h,w = rawImg.shape[:2]
    #imS = cv2.resize(rawImg, (w*3,h*3),interpolation = cv2.INTER_CUBIC)
    imS = rawImg
    gray = cv2.cvtColor(imS, cv2.COLOR_BGR2GRAY)
    
    if vid is None:
        vid = cv2.VideoWriter(outFile,fourcc, 30.0, (256,256))

    # Detect faces in the image
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=1,
        minSize=(150, 150),
        flags = cv2.CASCADE_SCALE_IMAGE
        )
    for (x, y, w, h) in faces:
        cv2.rectangle(gray, (x, y), (x+w, y+h), (0, 255, 0), 4)
        vid.write(gray)
        cv2.imshow('Face',gray)
        cv2.waitKey(30)
    if ct > 10:
        break
vid.release()
cv2.destroyAllWindows()

In [ ]:
cv2.destroyAllWindows()

In [ ]:
from numpy import mean,cov,cumsum,dot,linalg,size,flipud,argsort

def princomp(A,numpc=0):
    # computing eigenvalues and eigenvectors of covariance matrix
    M = (A-mean(A.T,axis=1)).T # subtract the mean (along columns)
    [latent,coeff] = linalg.eig(cov(M))
    p = size(coeff,axis=1)
    idx = argsort(latent) # sorting the eigenvalues
    idx = idx[::-1]       # in ascending order
    # sorting eigenvectors according to the sorted eigenvalues
    coeff = coeff[:,idx]
    latent = latent[idx] # sorting eigenvalues
    if numpc < p and numpc >= 0:
            coeff = coeff[:,range(numpc)] # cutting some PCs if needed
    score = dot(coeff.T,M) # projection of the data in the new space
    return coeff,score,latent

from pylab import imread,subplot,imshow,title,gray,figure,show,NullLocator
A = rawImg
A = mean(A,2) # to get a 2-D array
full_pc = size(A,axis=1) # numbers of all the principal components
i = 1
dist = []

step = 10
for numpc in range(0,full_pc+step,step): # 0 step*1 step*2 ... full_pc
    coeff, score, latent = princomp(A,numpc)
    Ar = dot(coeff,score).T+mean(A,axis=0) # image reconstruction
    # difference in Frobenius norm
    dist.append(linalg.norm(A-Ar,'fro'))
    # showing the pics reconstructed with less than 50 PCs
    if numpc <= 50:
        ax = subplot(2,3,i,frame_on=False)
        ax.xaxis.set_major_locator(NullLocator()) # remove ticks
        ax.yaxis.set_major_locator(NullLocator())
        i += 1 
        imshow(Ar)
        title('PCs # '+str(numpc))
        gray()

figure()
imshow(A)
title('numpc FULL')
gray()
show()

In [ ]:
A

In [ ]:
gray.shape

In [ ]:
import cv2
import numpy
import time
inDir = '/Users/abock/Desktop/faceMovie/'

# Detect faces in the image
cascPath = '/usr/local/Cellar/opencv/2.4.13.2/share/OpenCV/haarcascades/haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(cascPath)

for i in range(1,11):
    rawImg = cv2.imread(inDir + 'frame' + str(i) + '.jpeg')
    imS = rawImg
    faces = faceCascade.detectMultiScale(
        imS,
        scaleFactor=1.1,
        minNeighbors=1,
        minSize=(100, 100),
        flags = cv2.CASCADE_SCALE_IMAGE
        )
    for (x, y, w, h) in faces:
        cv2.rectangle(imS, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.imshow('Face',imS)
        cv2.waitKey(30)
time.sleep(3)
cv2.destroyAllWindows()

In [ ]:
cv2.destroyAllWindows()

In [ ]:
faces

In [ ]:
rawImg = cv2.imread('/Users/abock/aia_anger_561.png')
imS = rawImg
gray = cv2.cvtColor(imS, cv2.COLOR_BGR2GRAY)



faces = faceCascade.detectMultiScale(
    gray,
    scaleFactor=1.1,
    minNeighbors=1,
    minSize=(10, 10),
    flags = cv2.CASCADE_SCALE_IMAGE
    )
for (x, y, w, h) in faces:
    cv2.rectangle(gray, (x, y), (x+w, y+h), (0, 255, 0), 2)
    cv2.imshow('gray',gray)
    cv2.waitKey(30)
    
time.sleep(3)

In [ ]:
import cv2

cascPath = '/usr/local/Cellar/opencv/2.4.13.2/share/OpenCV/haarcascades/haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(cascPath)

rawImg = cv2.imread('/Users/abock/aia_anger_561.png')
imS = rawImg
gray = cv2.cvtColor(imS, cv2.COLOR_BGR2GRAY)
faces = faceCascade.detectMultiScale(
    gray,
    scaleFactor=1.1,
    minNeighbors=1,
    minSize=(10, 10),
    flags = cv2.CASCADE_SCALE_IMAGE
    )
for (x, y, w, h) in faces:
    cv2.rectangle(imS, (x, y), (x+w, y+h), (0, 255, 0), 4)
    cv2.imshow('Face',imS)
    cv2.waitKey(30)

# Record Movie

In [ ]:
import numpy as np
import cv2
import time
import os

outDir = '/Users/abock/Desktop/faceMovie/'
if not os.path.isdir(outDir):
    os.makedirs(outDir)

cap = cv2.VideoCapture(0)

ct = 0
while ct < 50:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Display the resulting frame
    cv2.imshow('frame',gray)
    cv2.waitKey(30)
    
    # save the frames
    ct = ct + 1
    outFrame = os.path.join(outDir, "frame" + str(ct) + ".jpeg")
    cv2.imwrite(outFrame,gray)
    
    # save a movie
    cv2.write(outVid,gray)

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

# Train model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, datasets

# import some data to play with
iris = datasets.load_iris()
X = iris.data[:, :2]  # we only take the first two features.
Y = iris.target

h = .02  # step size in the mesh

logreg = linear_model.LogisticRegression(C=1e5)

# we create an instance of Neighbours Classifier and fit the data.
logreg.fit(X, Y)

# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = logreg.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(1, figsize=(4, 3))
plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)

# Plot also the training points
plt.scatter(X[:, 0], X[:, 1], c=Y, edgecolors='k', cmap=plt.cm.Paired)
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')

plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xticks(())
plt.yticks(())

plt.show()

# Download images

In [ ]:
from bs4 import BeautifulSoup
import urllib2 
import urllib
import os

def saveURLimages(url,outDir):
    req = urllib2.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    html = urllib2.urlopen(req).read()
    soup = BeautifulSoup(html)
    # get images
    images = [img for img in soup.findAll('img')]
    print (str(len(images)) + " images found.")
    print 'Downloading images to: ' + outDir
    
    # download images
    image_links = [each.get('src') for each in images]
    #print(image_links)
    for each in image_links:
        if not each is None:
            if each[0:4] == 'http':
                #print(each)
                filename=each.split('/')[-1]
                urllib.urlretrieve(each,os.path.join(outDir,filename + '.jpg'))

### Google

In [ ]:
# Save happy faces (Google Images)
happyPerson = 'https://www.google.com/search?site=&tbm=isch&source=hp&biw=1680&bih=949&q=happy+person&oq=happy+person&gs_l=img.3..0l10.15311.17067.0.17213.12.7.0.5.5.0.82.450.7.7.0....0...1ac.1.64.img..0.12.461.FpA8-IvoCOk'
url = happyPerson
outDir = '/Users/abock/Desktop/HappyFaces'
if not os.path.isdir(outDir):
    os.makedirs(outDir)
saveURLimages(url,outDir)

# Save sad faces (Google Images)
sadPerson = 'https://www.google.com/search?site=&tbm=isch&source=hp&biw=840&bih=949&q=sad+person&oq=sad+person&gs_l=img.3..0l10.732.2268.0.2413.10.7.0.3.3.0.337.1010.0j1j2j1.4.0....0...1ac.1.64.img..3.7.1015.rkrdqlI1p-k'
url = sadPerson
outDir = '/Users/abock/Desktop/SadFaces'
if not os.path.isdir(outDir):
    os.makedirs(outDir)
saveURLimages(url,outDir)

### Bing

In [ ]:
# Save happy faces (Bing Images)
happyPerson = 'https://www.bing.com/images/search?q=happy%20person&qs=n&form=QBIR&pq=happy%20person&sc=8-12&sp=-1&sk='
url = happyPerson
outDir = '/Users/abock/Desktop/HappyFaces'
if not os.path.isdir(outDir):
    os.makedirs(outDir)
saveURLimages(url,outDir)

# Save sad faces (Bing Images)
sadPerson = 'https://www.bing.com/images/search?q=sad+person&qs=n&form=QBILPG&pq=sad+person&sc=8-8&sp=-1&sk='
url = sadPerson
outDir = '/Users/abock/Desktop/SadFaces'
if not os.path.isdir(outDir):
    os.makedirs(outDir)
saveURLimages(url,outDir)

### Happy Faces

In [ ]:
# Save happy faces (Yahoo Images)
outDir = '/Users/abock/Desktop/HappyFaces'
if not os.path.isdir(outDir):
    os.makedirs(outDir)
    
happyPerson = 'https://images.search.yahoo.com/search/images;_ylt=AwrTcXdqJYFYvE0A0juLuLkF?p=happy+person&ei=UTF-8&iscqry=&fr=sfp'
url = happyPerson
saveURLimages(url,outDir)

happyHumanFace = 'https://images.search.yahoo.com/search/images;_ylt=AwrTcXyAJoFYWHMApB6JzbkF;_ylu=X3oDMTBsZ29xY3ZzBHNlYwNzZWFyY2gEc2xrA2J1dHRvbg--;_ylc=X1MDOTYwNjI4NTcEX3IDMgRhY3RuA2NsawRiY2sDNmltaTE4bGM2N29ybyUyNmIlM0QzJTI2cyUzRDI5BGNzcmNwdmlkAzNzeFdhVEl3Tmk1cFdrRkZXR1BqZUFkSE5qVXVPQUFBQUFDdHdqNncEZnIDc2ZwBGZyMgNzYS1ncARncHJpZANLQm5lbEt4X1JOeUlzNXY5WC40cVRBBG10ZXN0aWQDbnVsbARuX3N1Z2cDMTAEb3JpZ2luA2ltYWdlcy5zZWFyY2gueWFob28uY29tBHBvcwMwBHBxc3RyAwRwcXN0cmwDBHFzdHJsAzE2BHF1ZXJ5A2hhcHB5IGh1bWFuIGZhY2UEdF9zdG1wAzE0ODQ4NTkwNzYEdnRlc3RpZANudWxs?gprid=KBnelKx_RNyIs5v9X.4qTA&pvid=3sxWaTIwNi5pWkFFWGPjeAdHNjUuOAAAAACtwj6w&p=happy+human+face&fr=sfp&fr2=sb-top-images.search.yahoo.com&ei=UTF-8&n=60&x=wrt'
url = happyHumanFace
saveURLimages(url,outDir)



### Sad Faces

In [ ]:
# Output directory
outDir = '/Users/abock/Desktop/SadFaces'
if not os.path.isdir(outDir):
    os.makedirs(outDir)
   
# Yahoo Images
sadPerson = 'https://images.search.yahoo.com/search/images;_ylt=AwrTcXZ6JYFYl68AvSGJzbkF;_ylu=X3oDMTBsZ29xY3ZzBHNlYwNzZWFyY2gEc2xrA2J1dHRvbg--;_ylc=X1MDOTYwNjI4NTcEX3IDMgRhY3RuA2NsawRiY2sDNmltaTE4bGM2N29ybyUyNmIlM0QzJTI2cyUzRDI5BGNzcmNwdmlkA3VBdHlfekl3Tmk1cFdrRkZXR1BqZUFDaE5qVXVPQUFBQUFDZUhiR3oEZnIDc2ZwBGZyMgNzYS1ncARncHJpZANnb3NOVG13V1RTdVc1MXA5ZmhMZGlBBG10ZXN0aWQDbnVsbARuX3N1Z2cDMTAEb3JpZ2luA2ltYWdlcy5zZWFyY2gueWFob28uY29tBHBvcwMwBHBxc3RyAwRwcXN0cmwDBHFzdHJsAzEwBHF1ZXJ5A3NhZCBwZXJzb24EdF9zdG1wAzE0ODQ4NTg3NjAEdnRlc3RpZANudWxs?gprid=gosNTmwWTSuW51p9fhLdiA&pvid=uAty_zIwNi5pWkFFWGPjeAChNjUuOAAAAACeHbGz&p=sad+person&fr=sfp&fr2=sb-top-images.search.yahoo.com&ei=UTF-8&n=60&x=wrt'
url = sadPerson
saveURLimages(url,outDir)

sadHumanFace = 'https://images.search.yahoo.com/search/images;_ylt=AwrTcXmHJYFY_lwACdOJzbkF;_ylu=X3oDMTBsZ29xY3ZzBHNlYwNzZWFyY2gEc2xrA2J1dHRvbg--;_ylc=X1MDOTYwNjI4NTcEX3IDMgRhY3RuA2NsawRiY2sDNmltaTE4bGM2N29ybyUyNmIlM0QzJTI2cyUzRDI5BGNzcmNwdmlkA2N3Q2RJREl3Tmk1cFdrRkZXR1BqZUFUVU5qVXVPQUFBQUFDZTZzVXEEZnIDc2ZwBGZyMgNzYS1ncARncHJpZANKZHp4ZFBPMVFfSzFlYlZTTkpycGhBBG10ZXN0aWQDbnVsbARuX3N1Z2cDMTAEb3JpZ2luA2ltYWdlcy5zZWFyY2gueWFob28uY29tBHBvcwMwBHBxc3RyAwRwcXN0cmwDBHFzdHJsAzE0BHF1ZXJ5A3NhZCBodW1hbiBmYWNlBHRfc3RtcAMxNDg0ODU5MDA5BHZ0ZXN0aWQDbnVsbA--?gprid=JdzxdPO1Q_K1ebVSNJrphA&pvid=cwCdIDIwNi5pWkFFWGPjeATUNjUuOAAAAACe6sUq&p=sad+human+face&fr=sfp&fr2=sb-top-images.search.yahoo.com&ei=UTF-8&n=60&x=wrt'
url = sadHumanFace
saveURLimages(url,outDir)